In [1]:
import os
import pickle
import numpy as np
from ase import io
from collections import defaultdict

In [2]:
xyz_files = os.listdir("xyzs")
final_files = ["xyzs/"+x for x in xyz_files if "final" in x]

In [3]:
logs_to_check = []
names2check = []
init_geoms = defaultdict()
for log in final_files:
    atoms_obj = io.read(log.replace("_final", "_init"))
    init_struct = atoms_obj.get_positions()
    final_struct = io.read(log).get_positions()
    atomic_nums = atoms_obj.get_atomic_numbers()
    origin_i = init_struct[np.argmax(atomic_nums)]
    origin_f = final_struct[np.argmax(atomic_nums)]
    init_distances = []
    final_distances = []
    for xyz_i, xyz_f in zip(init_struct, final_struct):
        init_distances.append(np.sqrt((xyz_i[0]-origin_i[0])**2+(xyz_i[1]-origin_i[1])**2+(xyz_i[2]-origin_i[2])**2))
        final_distances.append(np.sqrt((xyz_f[0]-origin_f[0])**2+(xyz_f[1]-origin_f[1])**2+(xyz_f[2]-origin_f[2])**2))
        
    num_neighbours = 6
    init_geoms[log.split("/")[-1]] = log.replace("_final", "_init")
    nearest_neighbours_i = np.argpartition(init_distances, num_neighbours)
    nearest_neighbours_i = [nn for nn in nearest_neighbours_i[:num_neighbours+1] if nn!=np.argmax(atomic_nums)]
    nearest_neighbours_f = np.argpartition(final_distances, num_neighbours)
    nearest_neighbours_f = [nn for nn in nearest_neighbours_f[:num_neighbours+1] if nn!=np.argmax(atomic_nums)]
    if set(nearest_neighbours_i)!=set(nearest_neighbours_f):
        print(nearest_neighbours_i)
        print(nearest_neighbours_f)
        print("**********************************")
        print("Atoms coordinating the metal have changed: {}".format(log))
        print("**********************************")
        logs_to_check.append(log)
        names2check.append(log.split("/")[-1])
    else:
        # all the indices are the same for init and final, just check the bond length change
        for index in nearest_neighbours_i:
            if final_distances[index] > 1.2*init_distances[index]:
                print(index)
                print("Bond distance 20% increase for {}".format(log))
                logs_to_check.append(log)
                names2check.append(log.split("/")[-1])

[18, 44, 25, 27, 3, 17]
[18, 44, 25, 27, 3, 26]
**********************************
Atoms coordinating the metal have changed: xyzs/Mn_oct_desorbed_5_OH_HS_final.xyz
**********************************
[17, 25, 3, 27, 44, 18]
[25, 18, 44, 27, 3, 26]
**********************************
Atoms coordinating the metal have changed: xyzs/Mn_oct_desorbed_4_OH_HS_final.xyz
**********************************
[15, 30, 32, 17, 16, 48]
[15, 48, 30, 17, 16, 31]
**********************************
Atoms coordinating the metal have changed: xyzs/Mn_3a1_1_OH_HS_final.xyz
**********************************
[26, 15, 1, 28, 17, 45]
[17, 45, 28, 1, 26, 27]
**********************************
Atoms coordinating the metal have changed: xyzs/Mn_3a1_11_OH_HS_final.xyz
**********************************
[17, 18, 3, 28, 30, 41]
[28, 17, 3, 18, 29, 41]
**********************************
Atoms coordinating the metal have changed: xyzs/Mn_3a1_18_OH_HS_final.xyz
**********************************
[17, 63, 15, 46, 16, 30

Having flagged these optimizations these simulations were checked manually to determine whether the geometry was truly wrong.  

In [4]:
# from checking these logfiles...
bad_oer_desc_catalysts = ["Mn_3a1_14", "Fe_41a_2_", "Mn_31a_7_", "Fe_32a_32", "Co_3a1_10", "Co_41a_1_", "Mn_31a_12",
                        "Mn_31a_11", "Mn_31a_15", "Mn_41a_2_", "Mn_31a_3_", "Mn_31a_8_", "Mn_3a1_15"]

In [5]:
from collections import defaultdict

desc_data = pickle.load(open("oer_desc_gibbs.p", 'rb'))
oer_desc_data = defaultdict()
for name in desc_data:
    if name not in bad_oer_desc_catalysts and desc_data[name]["OH"]!=0:
        # the way this data structure was formulated means we need 
        # to transform it in this way to get the oer descriptor.
        for xyz_file in init_geoms:
            # find the xyz file in the init_geoms dictionary that matches the catalyst name 
            # by its appearance as a substring in the filename
            if name in xyz_file and "OH" in xyz_file:
                oer_desc_data[name] = (-desc_data[name]["OH"], init_geoms[xyz_file])
                break

In [6]:

pickle.dump(oer_desc_data, open("oer_desc_data.p", "wb"), protocol=2)